# Interpolasi

In [2]:
import pandas as pd

# Load dataset varietas (Profile Ideal)
df_varietas = pd.read_excel('./dataset/split_data/varietas_iklim.xlsx')

# Load dataset lahan (Profile Alternatif)
df_lahan = pd.read_excel('./dataset/split_data/lahan_iklim.xlsx')

# Function utama untuk interpolasi
def interpolasi(x, min_val, max_val):
    if min_val <= x <= max_val:
        return 5
    elif 0 <= x < min_val:
        return 1 + ((x - 0) / (min_val - 0)) * (5 - 1)
    elif max_val < x <= (min_val + max_val):
        return 5 + ((x - max_val) / ((min_val + max_val) - max_val)) * (1 - 5)
    else:
        return 1

# Function untuk menghitung interpolasi berdasarkan jenis varietas
def hitung_skor_varietas(row, jenis_varietas):
    # Mengambil data varietas berdasarkan jenis_varietas
    var_data = df_varietas[df_varietas['jenis_varietas'] == jenis_varietas].iloc[0]
    
    hasil = {
        'suhu': interpolasi(row['suhu_mean'], var_data['suhu_min'], var_data['suhu_max']),
        'curah_hujan': interpolasi(row['curah_hujan'], var_data['curah_hujan_min'], var_data['curah_hujan_max']),
        'kelembapan_udara': interpolasi(row['kelembapan_udara'], var_data['kelembapan_udara_min'], var_data['kelembapan_udara_max']),
        'penyinaran': interpolasi(row['penyinaran'], var_data['penyinaran_min'], var_data['penyinaran_max'])
    }
    
    return hasil

# penggunaan untuk varietas 
def hitung_skor_V1(row):
    return hitung_skor_varietas(row, 'V1')

def hitung_skor_V2(row):
    return hitung_skor_varietas(row, 'V2')

def hitung_skor_V3(row):
    return hitung_skor_varietas(row, 'V3')

def hitung_skor_V4(row):
    return hitung_skor_varietas(row, 'V4')

def hitung_skor_V5(row):
    return hitung_skor_varietas(row, 'V5')




# Bobot Utama

In [3]:
bobot_utama = pd.read_excel('./dataset/bobot_ahp/parameter_utama.xlsx')
bobot_utama_dict = bobot_utama.set_index('Label')['Weight'].to_dict()
bobot_utama_dict

{'iklim': 0.4053875480953168,
 'topografi': 0.1139723667233914,
 'tanah': 0.4806400851812919}

### BOBOT AHP Parameter Iklim

In [4]:
bobot_iklim = pd.read_excel('./dataset/bobot_ahp/parameter_iklim.xlsx')
bobot_iklim_dict = bobot_iklim.set_index('Label')['Weight'].to_dict()
bobot_iklim_dict

{'suhu': 0.2117957615233342,
 'curah_hujan': 0.5251582768815123,
 'kelembapan_udara': 0.05125020007181938,
 'penyinaran': 0.2117957615233342}

## Varietas 1

In [5]:
lahan_scores = df_lahan.apply(hitung_skor_V1, axis=1)

# Menampilkan DataFrame hasil dengan nama lahan dan skor interpolasi
lahan_iklim_V1 = pd.concat([df_lahan['nama_lahan'], lahan_scores.apply(pd.Series)], axis=1)

lahan_iklim_V1

,nama_lahan,suhu,curah_hujan,kelembapan_udara,penyinaran
0,Bakung,5.0,5.0000,5.0,5.0
1,Binangun,5.0,5.0000,5.0,5.0
2,Doko,5.0,1.0000,5.0,5.0
3,Gandusari,5.0,1.0000,5.0,5.0
4,Garum,5.0,1.0000,5.0,5.0
5,Kademangan,5.0,3.5296,5.0,5.0
6,Kanigoro,5.0,5.0000,5.0,5.0
7,Kesamben,5.0,1.9368,5.0,5.0
8,Nglegok,5.0,1.0000,5.0,5.0
9,Panggungrejo,5.0,5.0000,5.0,5.0


### SUM (Perkalian skor V1 dengan bobot AHP)

In [6]:
skor_iklim_V1 = lahan_iklim_V1.copy()

# Melakukan perkalian setiap kolom dengan bobot yang sesuai
for kolom in ['suhu', 'curah_hujan', 'kelembapan_udara', 'penyinaran']:
    skor_iklim_V1[kolom] = skor_iklim_V1[kolom] * bobot_iklim_dict[kolom]

skor_iklim_V1['total_skor'] = skor_iklim_V1[['suhu', 'curah_hujan', 'kelembapan_udara', 'penyinaran']].sum(axis=1)

skor_iklim_V1


,nama_lahan,suhu,curah_hujan,kelembapan_udara,penyinaran,total_skor
0,Bakung,1.058979,2.625791,0.256251,1.058979,5.000000
1,Binangun,1.058979,2.625791,0.256251,1.058979,5.000000
2,Doko,1.058979,0.525158,0.256251,1.058979,2.899367
3,Gandusari,1.058979,0.525158,0.256251,1.058979,2.899367
4,Garum,1.058979,0.525158,0.256251,1.058979,2.899367
5,Kademangan,1.058979,1.853599,0.256251,1.058979,4.227807
6,Kanigoro,1.058979,2.625791,0.256251,1.058979,5.000000
7,Kesamben,1.058979,1.017127,0.256251,1.058979,3.391335
8,Nglegok,1.058979,0.525158,0.256251,1.058979,2.899367
9,Panggungrejo,1.058979,2.625791,0.256251,1.058979,5.000000


### Perkalian skor total iklim V1 dengan bobot AHP tanah

In [7]:

skor_total_iklim_V1 = skor_iklim_V1.copy()
skor_total_iklim_V1['total_skor_iklim_V1'] = skor_total_iklim_V1['total_skor'] * bobot_utama_dict['iklim']
skor_total_iklim_V1


,nama_lahan,suhu,curah_hujan,kelembapan_udara,penyinaran,total_skor,total_skor_iklim_V1
0,Bakung,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
1,Binangun,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
2,Doko,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
3,Gandusari,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
4,Garum,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
5,Kademangan,1.058979,1.853599,0.256251,1.058979,4.227807,1.713900
6,Kanigoro,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
7,Kesamben,1.058979,1.017127,0.256251,1.058979,3.391335,1.374805
8,Nglegok,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
9,Panggungrejo,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938


## Varietas 2

In [9]:
lahan_scores = df_lahan.apply(hitung_skor_V2, axis=1)

# Menampilkan DataFrame hasil dengan nama lahan dan skor interpolasi
lahan_iklim_V2 = pd.concat([df_lahan['nama_lahan'], lahan_scores.apply(pd.Series)], axis=1)

lahan_iklim_V2

,nama_lahan,suhu,curah_hujan,kelembapan_udara,penyinaran
0,Bakung,5.0,5.0000,5.0,5.0
1,Binangun,5.0,5.0000,5.0,5.0
2,Doko,5.0,1.0000,5.0,5.0
3,Gandusari,5.0,1.0000,5.0,5.0
4,Garum,5.0,1.0000,5.0,5.0
5,Kademangan,5.0,3.5296,5.0,5.0
6,Kanigoro,5.0,5.0000,5.0,5.0
7,Kesamben,5.0,1.9368,5.0,5.0
8,Nglegok,5.0,1.0000,5.0,5.0
9,Panggungrejo,5.0,5.0000,5.0,5.0


### SUM (perkalian skor V2 dengan bobot AHP )

In [10]:
skor_iklim_V2 = lahan_iklim_V2.copy()

# Melakukan perkalian setiap kolom dengan bobot yang sesuai
for kolom in ['suhu', 'curah_hujan', 'kelembapan_udara', 'penyinaran']:
    skor_iklim_V2[kolom] = skor_iklim_V2[kolom] * bobot_iklim_dict[kolom]

skor_iklim_V2['total_skor'] = skor_iklim_V2[['suhu', 'curah_hujan', 'kelembapan_udara', 'penyinaran']].sum(axis=1)

skor_iklim_V2


,nama_lahan,suhu,curah_hujan,kelembapan_udara,penyinaran,total_skor
0,Bakung,1.058979,2.625791,0.256251,1.058979,5.000000
1,Binangun,1.058979,2.625791,0.256251,1.058979,5.000000
2,Doko,1.058979,0.525158,0.256251,1.058979,2.899367
3,Gandusari,1.058979,0.525158,0.256251,1.058979,2.899367
4,Garum,1.058979,0.525158,0.256251,1.058979,2.899367
5,Kademangan,1.058979,1.853599,0.256251,1.058979,4.227807
6,Kanigoro,1.058979,2.625791,0.256251,1.058979,5.000000
7,Kesamben,1.058979,1.017127,0.256251,1.058979,3.391335
8,Nglegok,1.058979,0.525158,0.256251,1.058979,2.899367
9,Panggungrejo,1.058979,2.625791,0.256251,1.058979,5.000000


### Perkalian skor total iklim V2 dengan bobot AHP tanah

In [11]:

skor_total_iklim_V2 = skor_iklim_V2.copy()
skor_total_iklim_V2['total_skor_iklim_V2'] = skor_total_iklim_V2['total_skor'] * bobot_utama_dict['iklim']
skor_total_iklim_V2


,nama_lahan,suhu,curah_hujan,kelembapan_udara,penyinaran,total_skor,total_skor_iklim_V2
0,Bakung,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
1,Binangun,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
2,Doko,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
3,Gandusari,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
4,Garum,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
5,Kademangan,1.058979,1.853599,0.256251,1.058979,4.227807,1.713900
6,Kanigoro,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
7,Kesamben,1.058979,1.017127,0.256251,1.058979,3.391335,1.374805
8,Nglegok,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
9,Panggungrejo,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938


## Varietas V3

In [12]:
lahan_scores = df_lahan.apply(hitung_skor_V3, axis=1)

# Menampilkan DataFrame hasil dengan nama lahan dan skor interpolasi
lahan_iklim_V3 = pd.concat([df_lahan['nama_lahan'], lahan_scores.apply(pd.Series)], axis=1)

lahan_iklim_V3

,nama_lahan,suhu,curah_hujan,kelembapan_udara,penyinaran
0,Bakung,5.0,5.0000,5.0,5.0
1,Binangun,5.0,5.0000,5.0,5.0
2,Doko,5.0,1.0000,5.0,5.0
3,Gandusari,5.0,1.0000,5.0,5.0
4,Garum,5.0,1.0000,5.0,5.0
5,Kademangan,5.0,3.5296,5.0,5.0
6,Kanigoro,5.0,5.0000,5.0,5.0
7,Kesamben,5.0,1.9368,5.0,5.0
8,Nglegok,5.0,1.0000,5.0,5.0
9,Panggungrejo,5.0,5.0000,5.0,5.0


### SUM (Perkalian skor V3 dengan bobot AHP)

In [13]:
skor_iklim_V3 = lahan_iklim_V3.copy()

# Melakukan perkalian setiap kolom dengan bobot yang sesuai
for kolom in ['suhu', 'curah_hujan', 'kelembapan_udara', 'penyinaran']:
    skor_iklim_V3[kolom] = skor_iklim_V3[kolom] * bobot_iklim_dict[kolom]

skor_iklim_V3['total_skor'] = skor_iklim_V3[['suhu', 'curah_hujan', 'kelembapan_udara', 'penyinaran']].sum(axis=1)

skor_iklim_V3


,nama_lahan,suhu,curah_hujan,kelembapan_udara,penyinaran,total_skor
0,Bakung,1.058979,2.625791,0.256251,1.058979,5.000000
1,Binangun,1.058979,2.625791,0.256251,1.058979,5.000000
2,Doko,1.058979,0.525158,0.256251,1.058979,2.899367
3,Gandusari,1.058979,0.525158,0.256251,1.058979,2.899367
4,Garum,1.058979,0.525158,0.256251,1.058979,2.899367
5,Kademangan,1.058979,1.853599,0.256251,1.058979,4.227807
6,Kanigoro,1.058979,2.625791,0.256251,1.058979,5.000000
7,Kesamben,1.058979,1.017127,0.256251,1.058979,3.391335
8,Nglegok,1.058979,0.525158,0.256251,1.058979,2.899367
9,Panggungrejo,1.058979,2.625791,0.256251,1.058979,5.000000


### Perkalian skor total iklim V3 dengan bobot AHP tanah

In [14]:

skor_total_iklim_V3 = skor_iklim_V3.copy()
skor_total_iklim_V3['total_skor_iklim_V3'] = skor_total_iklim_V3['total_skor'] * bobot_utama_dict['iklim']
skor_total_iklim_V3


,nama_lahan,suhu,curah_hujan,kelembapan_udara,penyinaran,total_skor,total_skor_iklim_V3
0,Bakung,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
1,Binangun,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
2,Doko,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
3,Gandusari,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
4,Garum,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
5,Kademangan,1.058979,1.853599,0.256251,1.058979,4.227807,1.713900
6,Kanigoro,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
7,Kesamben,1.058979,1.017127,0.256251,1.058979,3.391335,1.374805
8,Nglegok,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
9,Panggungrejo,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938


## Varietas V4

In [15]:
lahan_scores = df_lahan.apply(hitung_skor_V4, axis=1)

# Menampilkan DataFrame hasil dengan nama lahan dan skor interpolasi
lahan_iklim_V4 = pd.concat([df_lahan['nama_lahan'], lahan_scores.apply(pd.Series)], axis=1)

lahan_iklim_V4

,nama_lahan,suhu,curah_hujan,kelembapan_udara,penyinaran
0,Bakung,5.0,5.0000,5.0,5.0
1,Binangun,5.0,5.0000,5.0,5.0
2,Doko,5.0,1.0000,5.0,5.0
3,Gandusari,5.0,1.0000,5.0,5.0
4,Garum,5.0,1.0000,5.0,5.0
5,Kademangan,5.0,3.5296,5.0,5.0
6,Kanigoro,5.0,5.0000,5.0,5.0
7,Kesamben,5.0,1.9368,5.0,5.0
8,Nglegok,5.0,1.0000,5.0,5.0
9,Panggungrejo,5.0,5.0000,5.0,5.0


### SUM (Perkalian skor V4 dengan bobot AHP)

In [16]:
skor_iklim_V4 = lahan_iklim_V4.copy()

# Melakukan perkalian setiap kolom dengan bobot yang sesuai
for kolom in ['suhu', 'curah_hujan', 'kelembapan_udara', 'penyinaran']:
    skor_iklim_V4[kolom] = skor_iklim_V4[kolom] * bobot_iklim_dict[kolom]

skor_iklim_V4['total_skor'] = skor_iklim_V4[['suhu', 'curah_hujan', 'kelembapan_udara', 'penyinaran']].sum(axis=1)

skor_iklim_V4


,nama_lahan,suhu,curah_hujan,kelembapan_udara,penyinaran,total_skor
0,Bakung,1.058979,2.625791,0.256251,1.058979,5.000000
1,Binangun,1.058979,2.625791,0.256251,1.058979,5.000000
2,Doko,1.058979,0.525158,0.256251,1.058979,2.899367
3,Gandusari,1.058979,0.525158,0.256251,1.058979,2.899367
4,Garum,1.058979,0.525158,0.256251,1.058979,2.899367
5,Kademangan,1.058979,1.853599,0.256251,1.058979,4.227807
6,Kanigoro,1.058979,2.625791,0.256251,1.058979,5.000000
7,Kesamben,1.058979,1.017127,0.256251,1.058979,3.391335
8,Nglegok,1.058979,0.525158,0.256251,1.058979,2.899367
9,Panggungrejo,1.058979,2.625791,0.256251,1.058979,5.000000


### Perkalian skor total iklim V4 dengan bobot AHP tanah

In [17]:

skor_total_iklim_V4 = skor_iklim_V4.copy()
skor_total_iklim_V4['total_skor_iklim_V4'] = skor_total_iklim_V4['total_skor'] * bobot_utama_dict['iklim']
skor_total_iklim_V4


,nama_lahan,suhu,curah_hujan,kelembapan_udara,penyinaran,total_skor,total_skor_iklim_V4
0,Bakung,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
1,Binangun,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
2,Doko,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
3,Gandusari,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
4,Garum,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
5,Kademangan,1.058979,1.853599,0.256251,1.058979,4.227807,1.713900
6,Kanigoro,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
7,Kesamben,1.058979,1.017127,0.256251,1.058979,3.391335,1.374805
8,Nglegok,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
9,Panggungrejo,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938


## Varietas 5

In [18]:
lahan_scores = df_lahan.apply(hitung_skor_V5, axis=1)

# Menampilkan DataFrame hasil dengan nama lahan dan skor interpolasi
lahan_iklim_V5 = pd.concat([df_lahan['nama_lahan'], lahan_scores.apply(pd.Series)], axis=1)

lahan_iklim_V5

,nama_lahan,suhu,curah_hujan,kelembapan_udara,penyinaran
0,Bakung,5.0,5.0000,5.0,5.0
1,Binangun,5.0,5.0000,5.0,5.0
2,Doko,5.0,1.0000,5.0,5.0
3,Gandusari,5.0,1.0000,5.0,5.0
4,Garum,5.0,1.0000,5.0,5.0
5,Kademangan,5.0,3.5296,5.0,5.0
6,Kanigoro,5.0,5.0000,5.0,5.0
7,Kesamben,5.0,1.9368,5.0,5.0
8,Nglegok,5.0,1.0000,5.0,5.0
9,Panggungrejo,5.0,5.0000,5.0,5.0


### SUM (Perkalian skor V5 dengan bobot AHP)

In [19]:
skor_iklim_V5 = lahan_iklim_V5.copy()

# Melakukan perkalian setiap kolom dengan bobot yang sesuai
for kolom in ['suhu', 'curah_hujan', 'kelembapan_udara', 'penyinaran']:
    skor_iklim_V5[kolom] = skor_iklim_V5[kolom] * bobot_iklim_dict[kolom]

skor_iklim_V5['total_skor'] = skor_iklim_V5[['suhu', 'curah_hujan', 'kelembapan_udara', 'penyinaran']].sum(axis=1)

skor_iklim_V5


,nama_lahan,suhu,curah_hujan,kelembapan_udara,penyinaran,total_skor
0,Bakung,1.058979,2.625791,0.256251,1.058979,5.000000
1,Binangun,1.058979,2.625791,0.256251,1.058979,5.000000
2,Doko,1.058979,0.525158,0.256251,1.058979,2.899367
3,Gandusari,1.058979,0.525158,0.256251,1.058979,2.899367
4,Garum,1.058979,0.525158,0.256251,1.058979,2.899367
5,Kademangan,1.058979,1.853599,0.256251,1.058979,4.227807
6,Kanigoro,1.058979,2.625791,0.256251,1.058979,5.000000
7,Kesamben,1.058979,1.017127,0.256251,1.058979,3.391335
8,Nglegok,1.058979,0.525158,0.256251,1.058979,2.899367
9,Panggungrejo,1.058979,2.625791,0.256251,1.058979,5.000000


### Perkalian skor total iklim V5 dengan bobot AHP tanah

In [20]:

skor_total_iklim_V5 = skor_iklim_V5.copy()
skor_total_iklim_V5['total_skor_iklim_V5'] = skor_total_iklim_V5['total_skor'] * bobot_utama_dict['iklim']
skor_total_iklim_V5


,nama_lahan,suhu,curah_hujan,kelembapan_udara,penyinaran,total_skor,total_skor_iklim_V5
0,Bakung,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
1,Binangun,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
2,Doko,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
3,Gandusari,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
4,Garum,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
5,Kademangan,1.058979,1.853599,0.256251,1.058979,4.227807,1.713900
6,Kanigoro,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
7,Kesamben,1.058979,1.017127,0.256251,1.058979,3.391335,1.374805
8,Nglegok,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
9,Panggungrejo,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
